In [8]:
# bu çalışıyor
# kendi yazdığım kod

import os
os.environ ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import torch
import mediapipe as mp
import time


cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        cx = int((x1+x2)/2)
        cy = int((y1+y2)/2)

        if class_id == vehicle_id or class_id == vehicle_id2 :
            
            class_name = model.names[class_id] 
            cv2.rectangle(annotated_frame, (x1,y1), (x2,y2), (0,255,0), 2)

            text = "ID:{} {}".format(track_id, class_name)
            cv2.putText(annotated_frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

            track = track_history[track_id]
            track.append((cx, cy))
            
            if len(track) > 15:
                track.pop(0)

            points = np.hstack(track).astype("int32").reshape((-1,1,2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0,255,0), thickness=2)

    # el ile keşisen obje
    detected_objects = []  # Her döngüde sıfırlanmalı
    
    # kesişim kontrolü
    for hand_box in hand_boxes:
        for box in bboxes:
            x1, y1, x2, y2, track_id, score, class_id = box
            x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

            if class_id == vehicle_id or class_id == vehicle_id2: 
                class_name = model.names[class_id]
                if score > 0.5:
                    if (x1 < hand_box[2] and x2 > hand_box[0] and
                        y1 < hand_box[3] and y2 > hand_box[1]):
                        detected_objects.append(class_name)

    # temas eden obje varsa
    if detected_objects:
        y_offset = 50
        for obj_name in detected_objects:
            # Çerçeve üzerine yazdırma
            cv2.putText(annotated_frame, f'El ile Temas Eden Obje: {obj_name}', (10, y_offset), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 30  # Her yazı için alt satıra geç


    # FPS hesaplama
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime != 0 else 0
    pTime = cTime

    
    cv2.imshow("Object Tracking", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    

In [ ]:
# alternatif

for hand_box in hand_boxes:
        for box in bboxes:
            x1, y1, x2, y2, track_id, score, class_id = box
            
            if class_id == vehicle_id or class_id == vehicle_id2: 
                class_name = model.names[class_id]
                print(f"Object Coordinates: {x1, y1, x2, y2}")
                print(f"Hand Box Coordinates: {hand_box[2], hand_box[3], hand_box[0], hand_box[1]}")
                
                if True :
                    x_intersection = max(0, min(x2, hand_box[2]) - max(x1, hand_box[0]))
                    y_intersection = max(0, min(y2, hand_box[3]) - max(y1, hand_box[1]))
                
                    if x_intersection > 0 and y_intersection > 0:
                        detected_objects.append(class_name)
                        print(class_name + " temas etti")




In [7]:
# hızları aynı olduğunda aynı diye yazıyor 

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import torch
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# elin önceki pozisyonlarını saklamak için 
hand_tracks = defaultdict(lambda: deque(maxlen=15)) 

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

# HIZ
# Nesnenin hızı için bir sözlük
object_speeds = defaultdict(float)
# Elin hızı için bir sözlük
hand_speeds = defaultdict(float)


while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        # enumerate eklendi hız için
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # Elin merkez koordinatlarını hesaplama
            hand_cx = (x_min + x_max) // 2
            hand_cy = (y_min + y_max) // 2
            
            # Elin izini güncelleme
            hand_tracks[idx].append((hand_cx, hand_cy))
            
            # Hız hesaplama
            if len(hand_tracks[idx]) > 1:
                prev_x, prev_y = hand_tracks[idx][-2]
                distance = np.sqrt((hand_cx - prev_x) ** 2 + (hand_cy - prev_y) ** 2)  # Euclidean mesafe
                h_speed = distance * fps  # Hız (pixel/saniye)
                hand_speeds[idx] = h_speed  # Bu elin hızını güncelle

            # Hızı ekrana yazdırma
            hand_speed_text = f"Elin Hizi: {hand_speeds[idx]:.2f} px/s"
            cv2.putText(frame, hand_speed_text, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        if class_id == vehicle_id or class_id == vehicle_id2:
            
            class_name = model.names[class_id] 
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            text = "ID:{} {}".format(track_id, class_name)
            cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            track = track_history[track_id]
            track.append((cx, cy))
            
            if len(track) > 15:
                track.pop(0)

            points = np.hstack(track).astype("int32").reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

            # HIZ
            # Hız hesaplama
            if len(track) > 1:
                prev_x, prev_y = track[-2]
                distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)  # Euclidean mesafe
                speed = distance * fps  # Hız (pixel/saniye)
                object_speeds[track_id] = speed  # Bu nesnenin hızını güncelle

    

    # HIZ
    # Hızları ekrana yazdırma
    for track_id, speed in object_speeds.items():
        print(f"Track ID: {track_id} Hız: {speed:.2f} px/s")

    # El ile keşişen objelerin kontrolü
    detected_objects = []  # Her döngüde sıfırlanmalı
    
    coordinates = defaultdict(lambda: deque(maxlen=30))
    
    # kesişim kontrolü
    for hand_box in hand_boxes:
        for box in bboxes:
            x1, y1, x2, y2, track_id, score, class_id = box
            x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

            if class_id == vehicle_id or class_id == vehicle_id2: 
                class_name = model.names[class_id]
                if score > 0.5:
                    if (x1 < hand_box[2] and x2 > hand_box[0] and
                        y1 < hand_box[3] and y2 > hand_box[1]):
                        # Nesne ve el kutusu çakışıyorsa objeyi kaydet
                        detected_objects.append(class_name)

    # Temas eden objeleri ekrana yazdırma
    if detected_objects:
        y_offset = 50
        for obj_name in detected_objects:
            cv2.putText(annotated_frame, f'El ile Temas Eden Obje: {obj_name}', (10, y_offset), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 30

    # El ve obje hızlarını karşılaştır
    same_speed_objects = []
    for hand_id, hand_speed in hand_speeds.items():
        for obj_id, obj_speed in object_speeds.items():
            # tolerans 5.0
            if abs(hand_speed - obj_speed) <= 5.0 :
                same_speed_objects.append((hand_id, obj_id))

    # Sağ alt köşeye yazdırma
    if same_speed_objects:
        for hand_id, obj_id in same_speed_objects:
            cv2.putText(annotated_frame, 
                        f"El {hand_id} ve Obje {obj_id} Ayni Hizda", 
                        (10, frame.shape[0] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    
    # FPS hesaplama
    cTime = time.time()
    # İlk kare için sıfır bölme hatasını önlemek amacıyla pTime kontrolü
    if pTime != 0:
        fps = 1 / (cTime - pTime)
    else:
        fps = 0  # İlk karede FPS hesaplanmaz
    pTime = cTime

    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Track ID: 2 Hız: 239.29 px/s
Track ID: 2 Hız: 21.14 px/s
Track ID: 2 Hız: 10.38 px/s
Track ID: 2 Hız: 23.50 px/s
Track ID: 2 Hız: 19.29 px/s
Track ID: 2 Hız: 8.18 px/s
Track ID: 2 Hız: 20.44 px/s
Track ID: 2 Hız: 94.19 px/s
Track ID: 2 Hız: 0.00 px/s
Track ID: 2 Hız: 30.37 px/s
Track ID: 2 Hız: 0.00 px/s
Track ID: 2 Hız: 10.73 px/s
Track ID: 2 Hız: 9.88 px/s
Track ID: 2 Hız: 43.65 px/s
Track ID: 2 Hız: 0.00 px/s
Track ID: 2 Hız: 67.89 px/s
Track ID: 2 Hız: 72.85 px/s
Track ID: 2 Hız: 117.87 px/s
Track ID: 2 Hız: 115.32 px/s
Track ID: 2 Hız: 82.30 px/s
Track ID: 2 Hız: 0.00 px/s
Track ID: 2 Hız: 42.36 px/s
Track ID: 2 Hız: 84.66 px/s
Track ID: 2 Hız: 41.43 px/s
Track ID: 2 Hız: 37.65 px/s
Track ID: 2 Hız: 99.37 px/s
Track ID: 2 Hız: 49.89 px/s
Track ID: 2 Hız: 22.92 px/s
Track ID: 2 Hız: 49.20 px/s
Track ID: 2 Hız: 48.10 px/s
Track ID: 2 Hız: 61.68 px/s
Track ID: 2 Hız: 110.36 px/s
Track ID: 2 Hız: 19.39 px/s
Track ID: 2 Hız: 41.51 px/s
Track ID: 2 Hız: 43.97 px/s
Track ID: 2 Hız: 98.40

In [8]:

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import torch
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# elin önceki pozisyonlarını saklamak için 
hand_tracks = defaultdict(lambda: deque(maxlen=15)) 

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

# HIZ
# Nesnenin hızı için bir sözlük
object_speeds = defaultdict(float)
# Elin hızı için bir sözlük
hand_speeds = defaultdict(float)


# Tuttu durumunu takip eden sözlük
hold_status = defaultdict(bool)

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        # enumerate eklendi hız için
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # Elin merkez koordinatlarını hesaplama
            hand_cx = (x_min + x_max) // 2
            hand_cy = (y_min + y_max) // 2
            
            # Elin izini güncelleme
            hand_tracks[idx].append((hand_cx, hand_cy))
            
            # Hız hesaplama
            if len(hand_tracks[idx]) > 1:
                prev_x, prev_y = hand_tracks[idx][-2]
                distance = np.sqrt((hand_cx - prev_x) ** 2 + (hand_cy - prev_y) ** 2)  # Euclidean mesafe
                h_speed = distance * fps  # Hız (pixel/saniye)
                hand_speeds[idx] = h_speed  # Bu elin hızını güncelle


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    current_objects_in_contact = set()  # Temas halinde olan objeleri takip etmek için
    
    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        if class_id == vehicle_id or class_id == vehicle_id2:

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            
            class_name = model.names[class_id] 
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            text = "ID:{} {}".format(track_id, class_name)
            cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            track = track_history[track_id]
            track.append((cx, cy))
            
            if len(track) > 15:
                track.pop(0)

            points = np.hstack(track).astype("int32").reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

            # HIZ
            # Hız hesaplama
            if len(track) > 1:
                prev_x, prev_y = track[-2]
                distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)  # Euclidean mesafe
                speed = distance * fps  # Hız (pixel/saniye)
                object_speeds[track_id] = speed  # Bu nesnenin hızını güncelle

    

    # HIZ
    # Hızları ekrana yazdırma
    for track_id, speed in object_speeds.items():
        print(f"Track ID: {track_id} Hız: {speed:.2f} px/s")

    # El ile keşişen objelerin kontrolü
    detected_objects = []  # Her döngüde sıfırlanmalı
    
    coordinates = defaultdict(lambda: deque(maxlen=30))
    
    # kesişim kontrolü
    for hand_box in hand_boxes:
        for box in bboxes:
            x1, y1, x2, y2, track_id, score, class_id = box
            x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

            if class_id == vehicle_id or class_id == vehicle_id2: 
                class_name = model.names[class_id]
                if score > 0.5:
                    if (x1 < hand_box[2] and x2 > hand_box[0] and
                        y1 < hand_box[3] and y2 > hand_box[1]):
                        # Nesne ve el kutusu çakışıyorsa objeyi kaydet
                        detected_objects.append(class_name)
                        
                        # El ve obje hızlarını karşılaştır
                        same_speed_objects = []
                        for hand_id, hand_speed in hand_speeds.items():
                            for obj_id, obj_speed in object_speeds.items():
                                if hand_speed > 25.0 and obj_speed > 25.0:
                                    # tolerans 5.0
                                    if abs(hand_speed - obj_speed) <= 5.0 :
                                        same_speed_objects.append((hand_id, obj_id))
                                        # Nesne ve el kutusu çakışıyorsa objeyi kaydet
                        

    # Temas eden objeleri ekrana yazdırma
    if detected_objects:
        y_offset = 50
        for obj_name in detected_objects:
            cv2.putText(annotated_frame, f'El ile Temas Eden Obje: {obj_name}', (10, y_offset), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 30

    

    # Sağ alt köşeye tuttu diye yazdırma
    if same_speed_objects:
        for hand_id, obj_id in same_speed_objects:
            cv2.putText(annotated_frame, 
                        f"El {hand_id} , Obje {obj_id} tuttu", 
                        (10, frame.shape[0] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    
    # FPS hesaplama
    cTime = time.time()
    # İlk kare için sıfır bölme hatasını önlemek amacıyla pTime kontrolü
    if pTime != 0:
        fps = 1 / (cTime - pTime)
    else:
        fps = 0  # İlk karede FPS hesaplanmaz
    pTime = cTime

    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Track ID: 4 Hız: 219.45 px/s
Track ID: 4 Hız: 36.87 px/s
Track ID: 4 Hız: 88.35 px/s
Track ID: 4 Hız: 108.52 px/s
Track ID: 4 Hız: 12.39 px/s
Track ID: 4 Hız: 11.55 px/s
Track ID: 4 Hız: 23.98 px/s
Track ID: 4 Hız: 14.36 px/s
Track ID: 4 Hız: 10.48 px/s
Track ID: 4 Hız: 14.27 px/s
Track ID: 4 Hız: 19.83 px/s
Track ID: 4 Hız: 11.36 px/s
Track ID: 4 Hız: 10.66 px/s
Track ID: 4 Hız: 51.02 px/s
Track ID: 4 Hız: 93.18 px/s
Track ID: 4 Hız: 33.32 px/s
Track ID: 4 Hız: 81.88 px/s
Track ID: 4 Hız: 47.08 px/s
Track ID: 4 Hız: 101.01 px/s
Track ID: 4 Hız: 102.59 px/s
Track ID: 4 Hız: 14.92 px/s
Track ID: 4 Hız: 14.30 px/s
Track ID: 4 Hız: 10.42 px/s
Track ID: 4 Hız: 43.72 px/s
Track ID: 4 Hız: 30.50 px/s
Track ID: 4 Hız: 0.00 px/s
Track ID: 4 Hız: 129.85 px/s
Track ID: 4 Hız: 104.56 px/s
Track ID: 4 Hız: 84.11 px/s
Track ID: 4 Hız: 53.50 px/s
Track ID: 4 Hız: 23.91 px/s
Track ID: 4 Hız: 16.48 px/s
Track ID: 4 Hız: 145.81 px/s
Track ID: 4 Hız: 141.19 px/s
Track ID: 4 Hız: 141.19 px/s
Track ID: 4 

In [9]:
# tuttu yazıyor bu kendi koduna entegre edilecek

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import torch
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# elin önceki pozisyonlarını saklamak için 
hand_tracks = defaultdict(lambda: deque(maxlen=15)) 

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

# HIZ
# Nesnenin hızı için bir sözlük
object_speeds = defaultdict(float)
# Elin hızı için bir sözlük
hand_speeds = defaultdict(float)


# Tuttu durumunu takip eden sözlük
hold_status = defaultdict(bool)

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        # enumerate eklendi hız için
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # Elin merkez koordinatlarını hesaplama
            hand_cx = (x_min + x_max) // 2
            hand_cy = (y_min + y_max) // 2
            
            # Elin izini güncelleme
            hand_tracks[idx].append((hand_cx, hand_cy))
            
            # Hız hesaplama
            if len(hand_tracks[idx]) > 1:
                prev_x, prev_y = hand_tracks[idx][-2]
                distance = np.sqrt((hand_cx - prev_x) ** 2 + (hand_cy - prev_y) ** 2)  # Euclidean mesafe
                h_speed = distance * fps  # Hız (pixel/saniye)
                hand_speeds[idx] = h_speed  # Bu elin hızını güncelle


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    current_objects_in_contact = set()  # Temas halinde olan objeleri takip etmek için
    
    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        class_name = model.names[class_id]
        print(f"Algılanan nesne: {class_name} (ID: {track_id}, Score: {score:.2f})")
        
        if class_id in [vehicle_id, vehicle_id2]:
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            track = track_history[track_id]
            track.append((cx, cy))
            if len(track) > 15:
                track.pop(0)

            if len(track) > 1:
                prev_x, prev_y = track[-2]
                distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
                speed = distance * fps
                object_speeds[track_id] = speed

            # Kesişim kontrolü
            for hand_box in hand_boxes:
                if (x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]):
                    current_objects_in_contact.add(track_id)  # Obje temas halinde

                    # Eğer hızlar aynıysa "tuttu" durumunu işaretle
                    for hand_id, hand_speed in hand_speeds.items():
                        if abs(hand_speed - object_speeds[track_id]) <= 5.0:
                            hold_status[track_id] = True

    
    # "Tuttu" yazısını kontrol et ve ekrana yazdır
    for track_id, is_held in hold_status.items():
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(annotated_frame, 
                        f"El {track_id} objeyi tuttu", 
                        (10, frame.shape[0] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)
            print(f"El {track_id} objeyi tuttu!")
            

    # Teması kesilen objelerin durumunu sıfırla
    for track_id in list(hold_status.keys()):
        if track_id not in current_objects_in_contact:
            hold_status.pop(track_id)

    cv2.imshow("Object Tracking", annotated_frame)

    # FPS hesaplama
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime != 0 else 0
    pTime = cTime

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: person (ID: 1, Score: 0.87)
Algılanan nesne: person (ID: 1, Score: 0.84)
Algılanan nesne: person (ID: 1, Score: 0.87)
Algılanan nesne: person (ID: 1, Score: 0.86)
Algılanan nesne: person (ID: 1, Score: 0.87)
Algılanan nesne: person (ID: 1, Score: 0.87)
Algılanan nesne: bottle (ID: 3, Score: 0.82)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: bottle (ID: 3, Score: 0.79)
Algılanan nesne: person (ID: 1, Score: 0.91)
Algılanan nesne: bottle (ID: 3, Score: 0.82)
Algılanan nesne: person (ID: 1, Score: 0.91)
Algılanan nesne: bottle (ID: 3, Score: 0.89)
Algılanan nesne: person (ID: 1, Score: 0.91)
Algılanan nesne: bottle (ID: 3, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.91)
Algılanan 

In [12]:

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import torch
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# elin önceki pozisyonlarını saklamak için 
hand_tracks = defaultdict(lambda: deque(maxlen=15)) 

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

# HIZ
# Nesnenin hızı için bir sözlük
object_speeds = defaultdict(float)
# Elin hızı için bir sözlük
hand_speeds = defaultdict(float)


# Tuttu durumunu takip eden sözlük
hold_status = defaultdict(bool)

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        # enumerate eklendi hız için
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # Elin merkez koordinatlarını hesaplama
            hand_cx = (x_min + x_max) // 2
            hand_cy = (y_min + y_max) // 2
            
            # Elin izini güncelleme
            hand_tracks[idx].append((hand_cx, hand_cy))
            
            # Hız hesaplama
            if len(hand_tracks[idx]) > 1:
                prev_x, prev_y = hand_tracks[idx][-2]
                distance = np.sqrt((hand_cx - prev_x) ** 2 + (hand_cy - prev_y) ** 2)  # Euclidean mesafe
                h_speed = distance * fps  # Hız (pixel/saniye)
                hand_speeds[idx] = h_speed  # Bu elin hızını güncelle


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    current_objects_in_contact = set()  # Temas halinde olan objeleri takip etmek için
    
    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        class_name = model.names[class_id]
        print(f"Algılanan nesne: {class_name} (ID: {track_id}, Score: {score:.2f})")
        
        if class_id in [vehicle_id, vehicle_id2]:
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            track = track_history[track_id]
            track.append((cx, cy))
            if len(track) > 15:
                track.pop(0)

            if len(track) > 1:
                prev_x, prev_y = track[-2]
                distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
                speed = distance * fps
                object_speeds[track_id] = speed

            # Kesişim kontrolü
            for hand_box in hand_boxes:
                if (x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]):
                    current_objects_in_contact.add(track_id)  # Obje temas halinde

                    # Eğer hızlar aynıysa "tuttu" durumunu işaretle
                    for hand_id, hand_speed in hand_speeds.items():
                        if abs(hand_speed - object_speeds[track_id]) <= 5.0:
                            hold_status[track_id] = True

    
    # "Tuttu" yazısını kontrol et ve ekrana yazdır
    for track_id, is_held in hold_status.items():
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(annotated_frame, 
                        f"El {track_id} objeyi tuttu", 
                        (10, frame.shape[0] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)
            print(f"El {track_id} objeyi tuttu!")
            

    # Teması kesilen objelerin durumunu sıfırla
    for track_id in list(hold_status.keys()):
        if track_id not in current_objects_in_contact:
            # Obje bırakıldı
            print(f"El {track_id} objeyi bıraktı!")
            cv2.putText(annotated_frame, 
                        f"El {track_id} objeyi bıraktı", 
                        (10, frame.shape[0] - 30 - 20 * track_id), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (255, 0, 0), 2)
            hold_status.pop(track_id)  # Obje artık tutulmuyor

    cv2.imshow("Object Tracking", annotated_frame)

    # FPS hesaplama
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime != 0 else 0
    pTime = cTime

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Algılanan nesne: person (ID: 1, Score: 0.87)
Algılanan nesne: person (ID: 1, Score: 0.87)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: person (ID: 1, Score: 0.90)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: bottle (ID: 2, Score: 0.92)
Algılanan nesne: person (ID: 1, Score: 0.88)
Algılanan nesne: bottle (ID: 2, Score: 0.92)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: bottle (ID: 2, Score: 0.92)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: bottle (ID: 2, Score: 0.92)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: bottle (ID: 2, Score: 0.92)
Algılanan nesne: person (ID: 1, Score: 0.90)
Algılanan nesne: bottle (ID: 2, Score: 0.94)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan nesne: bottle (ID: 2, Score: 0.93)
Algılanan nesne: person (ID: 1, Score: 0.89)
Algılanan 

In [10]:
# birleştirmeyi denediğim bu bakarsın 


import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import torch
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# elin önceki pozisyonlarını saklamak için 
hand_tracks = defaultdict(lambda: deque(maxlen=15)) 

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

# HIZ
# Nesnenin hızı için bir sözlük
object_speeds = defaultdict(float)
# Elin hızı için bir sözlük
hand_speeds = defaultdict(float)


# Tuttu durumunu takip eden sözlük
hold_status = defaultdict(bool)

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        # enumerate eklendi hız için
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # Elin merkez koordinatlarını hesaplama
            hand_cx = (x_min + x_max) // 2
            hand_cy = (y_min + y_max) // 2
            
            # Elin izini güncelleme
            hand_tracks[idx].append((hand_cx, hand_cy))
            
            # Hız hesaplama
            if len(hand_tracks[idx]) > 1:
                prev_x, prev_y = hand_tracks[idx][-2]
                distance = np.sqrt((hand_cx - prev_x) ** 2 + (hand_cy - prev_y) ** 2)  # Euclidean mesafe
                h_speed = distance * fps  # Hız (pixel/saniye)
                hand_speeds[idx] = h_speed  # Bu elin hızını güncelle


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    current_objects_in_contact = set()  # Temas halinde olan objeleri takip etmek için
    
    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        class_name = model.names[class_id]
        
        if class_id == vehicle_id or class_id == vehicle_id2:

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            
            class_name = model.names[class_id] 
            
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            text = "ID:{} {}".format(track_id, class_name)
            cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            track = track_history[track_id]
            track.append((cx, cy))
            
            if len(track) > 15:
                track.pop(0)

            # points = np.hstack(track).astype("int32").reshape((-1, 1, 2))
            # cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

            # HIZ
            # Hız hesaplama
            if len(track) > 1:
                prev_x, prev_y = track[-2]
                distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)  # Euclidean mesafe
                speed = distance * fps  # Hız (pixel/saniye)
                object_speeds[track_id] = speed  # Bu nesnenin hızını güncelle

            # Kesişim kontrolü
            for hand_box in hand_boxes:
                if (x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]):
                    current_objects_in_contact.add(track_id)  # Obje temas halinde

                    # Eğer hızlar aynıysa "tuttu" durumunu işaretle
                    for hand_id, hand_speed in hand_speeds.items():
                        if hand_speed > 25.0:
                            if object_speeds[track_id] > 25.0:
                                if abs(hand_speed - object_speeds[track_id]) <= 5.0:
                                    hold_status[track_id] = True


    # HIZ
    # Hızları ekrana yazdırma
    # for track_id, speed in object_speeds.items():
    #     print(f"Track ID: {track_id} Hız: {speed:.2f} px/s")

    # El ile keşişen objelerin kontrolü
    detected_objects = []  # Her döngüde sıfırlanmalı
    
    coordinates = defaultdict(lambda: deque(maxlen=30))

    #######
    # # kesişim kontrolü
    # for hand_box in hand_boxes:
    #     for box in bboxes:
    #         x1, y1, x2, y2, track_id, score, class_id = box
    #         x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

    #         if class_id == vehicle_id or class_id == vehicle_id2: 
    #             class_name = model.names[class_id]
    #             if score > 0.5:
    #                 if (x1 < hand_box[2] and x2 > hand_box[0] and
    #                     y1 < hand_box[3] and y2 > hand_box[1]):
    #                     # Nesne ve el kutusu çakışıyorsa objeyi kaydet
    #                     detected_objects.append(class_name)
                        
    #                     # El ve obje hızlarını karşılaştır
    #                     #######
    #                     # same_speed_objects = []
    #                     # for hand_id, hand_speed in hand_speeds.items():
    #                     #     for obj_id, obj_speed in object_speeds.items():
    #                     #         if hand_speed < 5.0:
    #                     #             if object_speeds[track_id] < 5.0:
    #                     #                 # tolerans 5.0
    #                     #                 if abs(hand_speed - obj_speed) <= 5.0 :
    #                     #                     same_speed_objects.append((hand_id, obj_id))
    #                     #                     # Nesne ve el kutusu çakışıyorsa objeyi kaydet
    #                     ########
    
    

    # # Temas eden objeleri ekrana yazdırma
    # if detected_objects:
    #     y_offset = 50
    #     for obj_name in detected_objects:
    #         cv2.putText(annotated_frame, f'El ile Temas Eden Obje: {obj_name}', (10, y_offset), 
    #                     cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    #         y_offset += 30

    ############



    
    # hold_status --> aynı hız , is_held --> aynı hızda olduğunu söyleyen boolean değer
    # current_object_in_contact --> temas 
    # "Tuttu" yazısını kontrol et ve ekrana yazdır
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(annotated_frame, 
                        f"El {track_id} id'li {class_name} objesini tuttu", 
                        (10, frame.shape[0] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)
            print(f"El {track_id} id'li {class_name} objesini tuttu!")
            for track_id in list(hold_status.keys()):
                if track_id not in current_objects_in_contact:
                    cv2.putText(annotated_frame, 
                                f"El {track_id} id'li {class_name} objesini birakti", 
                                (10, frame.shape[0] - 40), 
                                cv2.FONT_HERSHEY_SIMPLEX, 
                                0.7, (0, 0, 255), 2)
                    print(f"El {track_id} id'li {class_name} objesini birakti!")
                    hold_status.pop(track_id)
    

            
    # # Sağ alt köşeye tuttu diye yazdırma
    # if same_speed_objects:
    #     for hand_id, obj_id in same_speed_objects:
    #         cv2.putText(annotated_frame, 
    #                     f"El {hand_id} , Obje {obj_id} tuttu", 
    #                     (10, frame.shape[0] - 10), 
    #                     cv2.FONT_HERSHEY_SIMPLEX, 
    #                     0.7, (0, 0, 255), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    
    # FPS hesaplama
    cTime = time.time()
    # İlk kare için sıfır bölme hatasını önlemek amacıyla pTime kontrolü
    if pTime != 0:
        fps = 1 / (cTime - pTime)
    else:
        fps = 0  # İlk karede FPS hesaplanmaz
    pTime = cTime

    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


El 3 id'li bottle objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'li couch objesini tuttu!
El 3 id'l

In [11]:
# son versiyondan bi önceki

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2 
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import torch
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])

# Elin önceki pozisyonlarını saklamak için
hand_tracks = defaultdict(lambda: deque(maxlen=15)) 

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

# HIZ
# Nesnenin hızı için bir sözlük
object_speeds = defaultdict(float)
# Elin hızı için bir sözlük
hand_speeds = defaultdict(float)

# Tuttu durumunu takip eden sözlük
hold_status = defaultdict(bool)

# Bırakılan objelerin zamanını kaydetmek için sözlük
release_timestamps = {}

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # Elin merkez koordinatlarını hesaplama
            hand_cx = (x_min + x_max) // 2
            hand_cy = (y_min + y_max) // 2
            
            # Elin izini güncelleme
            hand_tracks[idx].append((hand_cx, hand_cy))
            
            # Hız hesaplama
            if len(hand_tracks[idx]) > 1:
                prev_x, prev_y = hand_tracks[idx][-2]
                distance = np.sqrt((hand_cx - prev_x) ** 2 + (hand_cy - prev_y) ** 2)
                h_speed = distance * fps
                hand_speeds[idx] = h_speed

    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    current_objects_in_contact = set()  # Temas halinde olan objeleri takip etmek için
    track_id_to_class_name = {}

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = box
        x1, y1, x2, y2, track_id, class_id = map(int, [x1, y1, x2, y2, track_id, class_id])

        if class_id == vehicle_id or class_id == vehicle_id2:
            track_id_to_class_name[track_id] = model.names[class_id]
            class_name = track_id_to_class_name[track_id]
            
            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)
            
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"ID:{track_id} {class_name}"
            cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            track = track_history[track_id]
            track.append((cx, cy))
            
            if len(track) > 15:
                track.pop(0)

            # Hız hesaplama
            if len(track) > 1:
                prev_x, prev_y = track[-2]
                distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
                speed = distance * fps
                object_speeds[track_id] = speed

            # Kesişim kontrolü
            for hand_box in hand_boxes:
                if (x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]):
                    current_objects_in_contact.add(track_id)

                    for hand_id, hand_speed in hand_speeds.items():
                        if hand_speed > 20.0 and object_speeds[track_id] > 20.0:
                            if abs(hand_speed - object_speeds[track_id]) <= 5.0:
                                hold_status[track_id] = True

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(annotated_frame, 
                        f"El {track_id} id'li {track_id_to_class_name[track_id]} objesini tuttu", 
                        (10, frame.shape[0] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)

        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = time.time()
            hold_status[track_id] = False

    # "Bıraktı" mesajını belirli bir süre göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")  # Varsayılan değer eklendi
            cv2.putText(annotated_frame, 
                        f"El {track_id} id'li {class_name} objesini birakti", 
                        (10, frame.shape[0] - 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (0, 0, 255), 2)
        else:
            release_timestamps.pop(track_id)


    # FPS hesaplama
    cTime = time.time()
    if pTime != 0:
        fps = 1 / (cTime - pTime)
    else:
        fps = 0
    pTime = cTime

    cv2.imshow("Object Tracking", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
# son versiyon iyileştirilmiş hali daha okunabilir bir kod

"""
El ve nesnenin hızlarının 20'den büyük olması gerekiyor (piksel üzerinden hesaplama)
Hız farkı 5'den küçükse "Tuttu" mesajı yazılıyor
Boş sonuçlar için kontroller eklenildi
"""

# tanınacak nesneler için class id'ler önceden belirlenmiş olmasına rağmen başka objeler de dahil olmakta
# bunun için üstüne çok uğraşılmadı çünkü kendi modelimizde zaten sadece istenilen objeler tanınacak
# bu kodda temel oturtulmuştur sonrasında modelimiz geliştirildiğinde kendi modelimiz denenecektir

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izlerini tutar
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            hand_tracks[idx].append((hand_cx, hand_cy))
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
        if class_id in [vehicle_id, vehicle_id2]:
            track_id_to_class_name[track_id] = model.names[class_id]
            class_name = track_id_to_class_name[track_id]
            
            # Kutu çizme
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"ID:{track_id} {class_name}"
            cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Nesne izleme
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            track_history[track_id].append((cx, cy))
            object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

            # Temas kontrolü
            for hand_box in hand_boxes:
                if (
                    x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]
                ):
                    current_objects_in_contact.add(track_id)

                    # Tuttu durumunu güncelle
                    hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                    if (
                        hand_speed > 10.0 and object_speeds[track_id] > 10.0 and
                        abs(hand_speed - object_speeds[track_id]) <= 5.0
                    ):
                        hold_status[track_id] = True

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




In [6]:
# son versiyon iyileştirilmiş hali daha okunabilir bir kod 

"""
El ve nesnenin hızlarının 20'den büyük olması gerekiyor (piksel üzerinden hesaplama)
Hız farkı 5'den küçükse "Tuttu" mesajı yazılıyor
Boş sonuçlar için kontroller eklenildi
"""

# tanınacak nesneler için class id'ler önceden belirlenmiş olmasına rağmen başka objeler de dahil olmakta
# bunun için üstüne çok uğraşılmadı çünkü kendi modelimizde zaten sadece istenilen objeler tanınacak
# bu kodda temel oturtulmuştur sonrasında modelimiz geliştirildiğinde kendi modelimiz denenecektir

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izlerini tutar
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0


# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            hand_tracks[idx].append((hand_cx, hand_cy))
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}

    for box in bboxes:
        if len(box) == 6:
            x1, y1, x2, y2, score, class_id = map(int, box[:6])
            track_id = -1
        else:
            x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
        if class_id in [vehicle_id, vehicle_id2]:
            track_id_to_class_name[track_id] = model.names[class_id]
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            track_history[track_id].append((cx, cy))
            object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

            # Temas kontrolü
            for hand_box in hand_boxes:
                if (
                    x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]
                ):
                    current_objects_in_contact.add(track_id)
                    class_name = model.names[class_id] 

                    cv2.rectangle(annotated_frame, (x1,y1), (x2,y2), (0,255,0), 2)

                    text = "ID:{} {}".format(track_id, class_name)
                    cv2.putText(annotated_frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

                    
                    # Tuttu durumunu güncelle
                    hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                    if (
                        hand_speed > 10.0 and object_speeds[track_id] > 10.0 and
                        abs(hand_speed - object_speeds[track_id]) <= 5.0
                    ):
                        hold_status[track_id] = True

                    

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




In [13]:
# yukarıdakinin kuyruk eklenmiş hali

"""
DEĞİŞKENLER
- track_history = nesnelerin izleme geçmişini saklamak için (15 kare)
- hand_tracks = ellerin izleme geçmişini saklamak için (15 kare)
- cTime = şu anki zaman (FPS)
- pTime = Önceki karedeki zaman (FPS)
- object_speeds = nesnelerin hızlarını saklamak için (float)
- hand_speeds = ellerin hızlarını saklamak için (float)
- hold_status = hangi nesneler eller tarafından tutulduğu (bool)
- release_timestamps = nesnelerin ne zaman bırakıldığı
- hand_boxes = [] : tespit edilen ellerin kutularını saklamak için
- model.track() = takip
- bboxes = tespit edilen nesnelerin sınır kutuları
- current_objects_in_contact = set() : tekrarlanan öğeleri saklamaz, temas halindeki nesneleri saklamak için
  + track_id 'si bu kümeye eklenir
- track_id_to_class_name = {} : anahtar değer çiftleri
  + tespit edilen nesnenin track_id ile class_name ilişkilendirme 
"""

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: deque(maxlen=15))  # Nesne izleme kuyrukları
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izleme kuyrukları
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

def update_track(track_dict, key, cx, cy):
    """İzleme kuyruklarını günceller"""
    track_dict[key].append((cx, cy))

def draw_trail(frame, track, color):
    """İzleri çizer"""
    for i in range(1, len(track)):
        cv2.line(frame, track[i - 1], track[i], color, 2)

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            update_track(hand_tracks, idx, hand_cx, hand_cy)
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

            # El izi çizimi
            draw_trail(frame, list(hand_tracks[idx]), (0, 0, 0))

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}
    

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
        if class_id in [vehicle_id, vehicle_id2]:
            track_id_to_class_name[track_id] = model.names[class_id]
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            update_track(track_history, track_id, cx, cy)
            object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

            # İz çizimi
            draw_trail(annotated_frame, list(track_history[track_id]), (255, 255, 255))

            # Temas kontrolü
            for hand_box in hand_boxes:
                if (
                    x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]
                ):
                    current_objects_in_contact.add(track_id)
                    class_name = model.names[class_id] 

                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    text = "ID:{} {}".format(track_id, class_name)
                    cv2.putText(annotated_frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Tuttu durumunu güncelle
                    hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                    if (
                        hand_speed > 10.0 and object_speeds[track_id] > 10.0 and
                        abs(hand_speed - object_speeds[track_id]) <= 5.0
                    ):
                        hold_status[track_id] = True

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [7]:
# son versiyon iyileştirilmiş hali daha okunabilir bir kod
# sadece tuttu bıraktı

"""
El ve nesnenin hızlarının 20'den büyük olması gerekiyor (piksel üzerinden hesaplama)
Hız farkı 5'den küçükse "Tuttu" mesajı yazılıyor
Boş sonuçlar için kontroller eklenildi
"""

# tanınacak nesneler için class id'ler önceden belirlenmiş olmasına rağmen başka objeler de dahil olmakta
# bunun için üstüne çok uğraşılmadı çünkü kendi modelimizde zaten sadece istenilen objeler tanınacak
# bu kodda temel oturtulmuştur sonrasında modelimiz geliştirildiğinde kendi modelimiz denenecektir

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "yolov8n.pt"
model = YOLO(model_name)
vehicle_id = 39
vehicle_id2 = 67
track_history = defaultdict(lambda: [])
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izlerini tutar
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            hand_tracks[idx].append((hand_cx, hand_cy))
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
        if class_id in [vehicle_id, vehicle_id2]:
            track_id_to_class_name[track_id] = model.names[class_id]
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            track_history[track_id].append((cx, cy))
            object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

            # Temas kontrolü
            for hand_box in hand_boxes:
                if (
                    x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]
                ):
                    current_objects_in_contact.add(track_id)

                    # Tuttu durumunu güncelle
                    hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                    if (
                        hand_speed > 10.0 and object_speeds[track_id] > 10.0 and
                        abs(hand_speed - object_speeds[track_id]) <= 5.0
                    ):
                        hold_status[track_id] = True

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




In [14]:
# kendi modelimiz
# kuyruk eklenmiş hali

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "best.pt"
model = YOLO(model_name)
track_history = defaultdict(lambda: deque(maxlen=15))  # Nesne izleme kuyrukları
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izleme kuyrukları
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

def update_track(track_dict, key, cx, cy):
    """İzleme kuyruklarını günceller"""
    track_dict[key].append((cx, cy))

def draw_trail(frame, track, color):
    """İzleri çizer"""
    for i in range(1, len(track)):
        cv2.line(frame, track[i - 1], track[i], color, 2)

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            hand_tracks[idx].append((hand_cx, hand_cy))
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

            # El izi çizimi
            # draw_trail(frame, list(hand_tracks[idx]), (0, 0, 0))

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}
    

    for box in bboxes:
        # Box'un uzunluğuna göre track_id kontrolü
        if len(box) == 6:
            x1, y1, x2, y2, score, class_id = map(int, box[:6])
            track_id = -1  # Track ID mevcut değilse varsayılan değeri
        else:
            x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
        
        track_id_to_class_name[track_id] = model.names[class_id]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        track_history[track_id].append((cx, cy))
        object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

        # İz çizimi
        # draw_trail(annotated_frame, list(track_history[track_id]), (255, 255, 255))

        # Temas kontrolü
        for hand_box in hand_boxes:
            if (
                x1 < hand_box[2] and x2 > hand_box[0] and
                y1 < hand_box[3] and y2 > hand_box[1]
            ):
                current_objects_in_contact.add(track_id)
                class_name = model.names[class_id] 

                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                text = "ID:{} {}".format(track_id, class_name)
                cv2.putText(annotated_frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Tuttu durumunu güncelle
                hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                if (
                    hand_speed > 5.0 and object_speeds[track_id] > 5.0 and
                    abs(hand_speed - object_speeds[track_id]) <= 5.0
                ):
                    hold_status[track_id] = True


    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# kendi modelimiz 
# sadece tuttu bıraktı

"""
El ve nesnenin hızlarının 20'den büyük olması gerekiyor (piksel üzerinden hesaplama)
Hız farkı 5'den küçükse "Tuttu" mesajı yazılıyor
Boş sonuçlar için kontroller eklenildi
"""

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "best.pt"
model = YOLO(model_name)
track_history = defaultdict(lambda: [])
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izlerini tutar
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            hand_tracks[idx].append((hand_cx, hand_cy))
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}

    for box in bboxes:
        if len(box) == 6:
            x1, y1, x2, y2, score, class_id = map(int, box[:6])
            track_id = -1  # Track ID mevcut değilse varsayılan değeri
        else:
            x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
            
        track_id_to_class_name[track_id] = model.names[class_id]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        track_history[track_id].append((cx, cy))
        object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

        # Temas kontrolü
        for hand_box in hand_boxes:
            if (
                x1 < hand_box[2] and x2 > hand_box[0] and
                y1 < hand_box[3] and y2 > hand_box[1]
            ):
                current_objects_in_contact.add(track_id)

                # Tuttu durumunu güncelle
                hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                if (
                    hand_speed > 5.0 and object_speeds[track_id] > 1.0 and
                    abs(hand_speed - object_speeds[track_id]) <= 5.0
                ):
                    hold_status[track_id] = True

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [8]:
# kendi modelimiz 
# sadece tuttu bıraktı

"""
El ve nesnenin hızlarının 20'den büyük olması gerekiyor (piksel üzerinden hesaplama)
Hız farkı 5'den küçükse "Tuttu" mesajı yazılıyor
Boş sonuçlar için kontroller eklenildi
"""

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import mediapipe as mp
import time

# YOLO ve MediaPipe ayarları
model_name = "best.pt"
model = YOLO(model_name)
track_history = defaultdict(lambda: [])
hand_tracks = defaultdict(lambda: deque(maxlen=15))  # El izlerini tutar
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# Hız ve durum bilgileri
object_speeds = defaultdict(float)
hand_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # El tespiti
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for idx, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
            x_min = int(min([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([lm.x for lm in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([lm.y for lm in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))

            # El izlerini ve hızını güncelleme
            hand_cx, hand_cy = (x_min + x_max) // 2, (y_min + y_max) // 2
            hand_tracks[idx].append((hand_cx, hand_cy))
            hand_speeds[idx] = calculate_speed(hand_tracks[idx], fps)

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}

    for box in bboxes:
        x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])
        track_id_to_class_name[track_id] = model.names[class_id]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        track_history[track_id].append((cx, cy))
        object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

        # Temas kontrolü
        for hand_box in hand_boxes:
            if (
                x1 < hand_box[2] and x2 > hand_box[0] and
                y1 < hand_box[3] and y2 > hand_box[1]
            ):
                current_objects_in_contact.add(track_id)

                # Tuttu durumunu güncelle
                hand_speed = max(hand_speeds.values(), default=0)  # El hızını kontrol et
                if (
                    hand_speed > 10.0 and object_speeds[track_id] > 10.0 and
                    abs(hand_speed - object_speeds[track_id]) <= 5.0
                ):
                    hold_status[track_id] = True

    # "Tuttu" ve "Bıraktı" mesajlarını işleme
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id in current_objects_in_contact:
            cv2.putText(
                annotated_frame, 
                f"El {track_id} id'li {track_id_to_class_name.get(track_id, 'bilinmeyen obje')} objesini tuttu", 
                (10, frame.shape[0] - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.7, (0, 0, 255), 2
            )
        elif is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    # Bırakma mesajlarını göster
    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if not class_name == "bilinmeyen obje":
                cv2.putText(
                    annotated_frame, 
                    f"El {track_id} id'li {class_name} objesini birakti", 
                    (10, frame.shape[0] - 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.7, (0, 255, 255), 2
                )
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [15]:
# hocanın önerisiyle yapılan modele göre olan kod versiyon-1

# burada sadece ekrana çizdirme

import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import imutils
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
import torch
import mediapipe as mp
import time


cap = cv2.VideoCapture(0)
model_name = "best-eski.pt"
model = YOLO(model_name)

track_history = defaultdict(lambda: [])

# MediaPipe el tespiti
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5)

# FPS hesaplamak için değişkenler
cTime = 0
pTime = 0

while True:
    ret, frame = cap.read()
    
    if ret == False:
        break

    # Görüntüyü RGB formatına çevirme
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_hands = hands.process(img_rgb)
    
    # El tespiti
    hand_boxes = []
    if results_hands.multi_hand_landmarks:
        for hand_landmarks in results_hands.multi_hand_landmarks:
            mp.solutions.drawing_utils.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            x_min = int(min([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_min = int(min([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            x_max = int(max([landmark.x for landmark in hand_landmarks.landmark]) * frame.shape[1])
            y_max = int(max([landmark.y for landmark in hand_landmarks.landmark]) * frame.shape[0])
            hand_boxes.append((x_min, y_min, x_max, y_max))


    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist())

    for box in bboxes:
        # box, [x1, y1, x2, y2, track_id, score, class_id] formatında olmalıdır, ancak 6 değer alıyorsan bu durumu kontrol et.
        if len(box) == 7:
            x1, y1, x2, y2, track_id, score, class_id = box
        elif len(box) == 6:
            x1, y1, x2, y2, track_id, class_id = box
            score = 1.0  # Eğer skor yoksa, varsayılan olarak 1.0 veriyoruz.
        else:
            continue  # Diğer olasılıkları kontrol et ve geç

        # Koordinatları int'e dönüştür
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)

        class_name = model.names[class_id] 
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        text = "ID:{} {}".format(track_id, class_name)
        cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        track = track_history[track_id]
        track.append((cx, cy))

        if len(track) > 15:
            track.pop(0)

        points = np.hstack(track).astype("int32").reshape((-1, 1, 2))
        cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 255, 0), thickness=2)

    # El ile keşişen obje
    detected_objects = []  # Her döngüde sıfırlanmalı
    
    # Kesişim kontrolü
    for hand_box in hand_boxes:
        for box in bboxes:
            # box, [x1, y1, x2, y2, track_id, score, class_id] formatında olmalıdır, ancak 6 değer alıyorsan bu durumu kontrol et.
            if len(box) == 7:
                x1, y1, x2, y2, track_id, score, class_id = box
            elif len(box) == 6:
                x1, y1, x2, y2, track_id, class_id = box
                score = 1.0  # Eğer skor yoksa, varsayılan olarak 1.0 veriyoruz.
            else:
                continue  # Diğer olasılıkları kontrol et ve geç

            class_name = model.names[class_id]
            if score > 0.5:
                if (x1 < hand_box[2] and x2 > hand_box[0] and
                    y1 < hand_box[3] and y2 > hand_box[1]):
                    detected_objects.append(class_name)

    # Temas eden obje varsa
    if detected_objects:
        y_offset = 50
        for obj_name in detected_objects:
            # Çerçeve üzerine yazdırma
            cv2.putText(annotated_frame, f'El ile Temas Eden Obje: {obj_name}', (10, y_offset), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            y_offset += 30  # Her yazı için alt satıra geç

    # FPS hesaplama
    cTime = time.time()
    fps = 1 / (cTime - pTime) if pTime != 0 else 0
    pTime = cTime

    cv2.imshow("Object Tracking", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [16]:
# hocanın önerisinin yapılmaya çalışıldığı kod


import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict, deque
import time

# YOLO ayarları
model_name = "best.pt"
model = YOLO(model_name)

# FPS hesaplama için zaman değişkenleri
cTime, pTime = 0, 0

# İzleme ve hız bilgileri
object_speeds = defaultdict(float)
hold_status = defaultdict(bool)
release_timestamps = {}

# Anahtar sınıf isimleri
TARGET_CLASSES = {"sise": "Şişe", "telefon": "Telefon", "elSise": "El Şişe", "elTelefon": "El Telefon"}

# Fonksiyonlar
def calculate_fps(cTime, pTime):
    """FPS hesaplama fonksiyonu"""
    fps = 1 / max(cTime - pTime, 1e-6)
    return fps, cTime

def calculate_speed(track, fps):
    """Hız hesaplama fonksiyonu"""
    if len(track) > 1:
        prev_x, prev_y = track[-2]
        cx, cy = track[-1]
        distance = np.sqrt((cx - prev_x) ** 2 + (cy - prev_y) ** 2)
        return distance * fps
    return 0

# Ana döngü
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Nesne tespiti
    annotated_frame = frame.copy()
    results = model.track(annotated_frame, persist=True, verbose=False)[0]
    bboxes = np.array(results.boxes.data.tolist()) if results.boxes else []

    current_objects_in_contact = set()
    track_id_to_class_name = {}
    track_history = defaultdict(lambda: deque(maxlen=15))

    for box in bboxes:
        if len(box) == 6:
            x1, y1, x2, y2, score, class_id = map(int, box[:6])
            track_id = -1
        else:
            x1, y1, x2, y2, track_id, score, class_id = map(int, box[:7])

        class_name = model.names[class_id]
        if class_name not in TARGET_CLASSES:
            continue

        track_id_to_class_name[track_id] = class_name
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        track_history[track_id].append((cx, cy))
        object_speeds[track_id] = calculate_speed(track_history[track_id], fps)

        # Şişe veya telefon durumları
        if class_name in ["sise", "telefon"]:
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"ID:{track_id} {TARGET_CLASSES[class_name]}"
            cv2.putText(annotated_frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # El şişeyi veya telefonu tutuyor
        elif class_name in ["elsise", "eltelefon"]:
            current_objects_in_contact.add(track_id)
            hand_speed = object_speeds[track_id]
            status = "taşıyor" if hand_speed > 5 else "duruyor"
            cv2.putText(annotated_frame, f"El {TARGET_CLASSES[class_name]} {status}", (10, frame.shape[0] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Bırakma durumlarını kontrol et
    current_time = time.time()
    for track_id, is_held in list(hold_status.items()):
        if is_held and track_id not in current_objects_in_contact:
            release_timestamps[track_id] = current_time
            hold_status[track_id] = False

    for track_id, timestamp in list(release_timestamps.items()):
        if current_time - timestamp <= 1.0:
            class_name = track_id_to_class_name.get(track_id, "bilinmeyen obje")
            if class_name in ["elsise", "eltelefon"]:
                cv2.putText(annotated_frame, f"El {TARGET_CLASSES[class_name]} bıraktı", (10, frame.shape[0] - 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        else:
            release_timestamps.pop(track_id)

    # FPS hesaplama ve gösterim
    cTime = time.time()
    fps, pTime = calculate_fps(cTime, pTime)
    cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    cv2.imshow("Object Tracking", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
